**best score!!!**

# Setting

In [1]:
# 기본 라이브러리
import os
import gc
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Scikit-learn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, f1_score

# PyTorch & Lightning
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

# Transformers
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import FunnelTokenizerFast, FunnelModel
transformers.logging.set_verbosity_error()

# 기타
import re
from tqdm import tqdm
import warnings
import random
warnings.filterwarnings('ignore')

In [2]:
SEED = 826

def set_seeds(seed=SEED):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    pl.seed_everything(SEED)
    
set_seeds()

Seed set to 826


# models

In [3]:
model_dict = {
    "bert-kor-base": {
        "model_name": "kykim/bert-kor-base",
        "latent_dim": 384
    },
    "kc-electra": {
        "model_name": "beomi/KcELECTRA-base",
        "latent_dim": 384
    },
    "funnel-kor-base": {
        "model_name": "kykim/funnel-kor-base",
        "latent_dim": 384
    }
}

# 실제 모델의 출력은 768

# preprocessing

`-` **Data Augmentation**

In [4]:
train = pd.read_csv('train.csv')

df_train = train.copy()

In [18]:
train

,id,text,label
0,1,제동 저항을 25Ω으로 만들기 위해서는 여러 개의 저항을 병렬 또는 직렬로 연결해야...,1
1,2,웹디자인 레이아웃을 잡을 때는 주로 포토샵을 사용합니다. 포토샵은 디자인 시안을 만...,1
2,3,엑셀 2010 피벗 테이블에서 특정 필드만 표로 추출하는 방법은 매우 간단합니다. ...,1
3,4,안녕하세요! 최근 반려동물 관련 TV 프로그램을 자주 시청하면서 이 분야에서 일하고...,1
4,5,"경량철골 천장의 특성을 고려할 때, 현재 상황이 심각한 하자로 보기는 어렵습니다. ...",1
...,...,...,...
21971,21972,"""Have you ever done anything special for someo...",1
21972,21973,"엑셀의 경우, 일반적으로 출력 결과가 일치하면 채점에서 인정받을 수 있습니다. 그러...",1
21973,21974,"가장 많이 사용하는 수학적 원리는 ""등호(=)를 이용한 등식의 성질""과 ""사칙연산(...",1
21974,21975,안녕하세요. 서울교대 면접 준비를 해야되는지 걱정이신데.. 요즘 추세는 면접에서 ...,0


In [19]:
test

,id,text
0,1,머리가 가라앉고 힘이 없다면 뿌리쪽에 볼륨을 줘야 합니다. 보통 일반 롯트펌은 머리...
1,2,문제가 일어났을시 그것이 거짓일경우 팬들은 거짓말을 한 부분에 대해 대체적으로 용납...
2,3,예~~맹독버섯인 흰가시광대버섯입니다 위장계통과 신장계통의 중독을 일으킨답니다 버섯대...
3,4,설득의 심리학 정도 권해드려봅니다 궁금한점은 채택 후 1대1 질문 주세요 의견 ...
4,5,안녕하세요! 신정(1월 1일)에는 동대문 악세사리 시장의 대부분의 상가가 휴업합니다...
...,...,...
6509,6510,"SK BTV의 경우, 3년 계약이 완료된 후에는 재가입이 가능합니다. 계약 기간이 ..."
6510,6511,"대엽홍콩야자 / 주필란서스 밝은 듯한 반양지 정도에 배치해서 관리하시고, 물은 ..."
6511,6512,안녕하세요! 젤리츄 배송과 관련해서 도와드리겠습니다. 일반적으로 젤리츄의 배송은 ...
6512,6513,심리상담사 자격증은 현재 국가기관인 한국직업능력개발원에 민간자격증으로 운영되고 있습...


In [5]:
from transformers import AutoTokenizer
import pandas as pd

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("kykim/bert-kor-base", use_fast=True)

# max 길이 설정
max_len = 384
stride = 192

# token_length 계산
df_train["token_length"] = df_train["text"].apply(lambda x: len(tokenizer.tokenize(x)))
df_long = df_train[df_train["token_length"] >= max_len]

augmented_rows = []

for _, row in df_long.iterrows():
    text = row["text"]
    label = row["label"]

    tokens = tokenizer.tokenize(text)
    total_len = len(tokens)

    for i in range(0, total_len - max_len + 1, stride):
        chunk = tokens[i: i + max_len]
        chunk_ids = tokenizer.convert_tokens_to_ids(chunk)
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True)
        augmented_rows.append({"text": chunk_text, "label": label})

    # 마지막 조각 추가
    if (total_len - max_len) % stride != 0:
        last_chunk = tokens[-max_len:]
        last_ids = tokenizer.convert_tokens_to_ids(last_chunk)
        last_text = tokenizer.decode(last_ids, skip_special_tokens=True)
        augmented_rows.append({"text": last_text, "label": label})

# 데이터프레임 변환
df_augmented = pd.DataFrame(augmented_rows).reset_index().rename(columns={'index': 'id'})
df_augmented

,id,text,label
0,0,제동 저항을 만들기 위해서는 여러 개의 저항을 병렬 또는 직렬로 연결해야 합니다. ...,1
1,1,직렬 연결 * * : 저항 값이 더해집니다. ( r _ total = r1 + r2...,1
2,2,안녕하세요? 최고의 품질평가 전문기관 축산물품질평가원 입니다. 먹어도 됩니다. ( ...,0
3,3,"출혈로 인해 발생한다. 만약 난황에 혈반이 있다면, 배란시점에 출혈이 있었거나 난관...",0
4,4,안녕하세요 국가 인증 교육원 플래너 방연미 팀장입니다 편입 하신다는 질문에 답변드립...,0
...,...,...,...
5072,5072,학점은행제에 대해 도움을 드릴 수 있는 이지쌤입니다. 학점은행제를 통해 심리학과에서...,1
5073,5073,전기산업기사 응시 자격에 대한 질문을 드립니다. 저는 19살에 특성화 고등학교를 졸...,1
5074,5074,19살에 특성화 고등학교를 졸업한 후 바로 취업하여 군대 휴직을 포함해 생산직으로 ...,1
5075,5075,"항공기에는 다양한 핵심 부품이 있으며, 이들은 비행기의 안전성과 성능을 보장하는 데...",1


In [6]:
df_augmented['label'].value_counts()

label
0    3842
1    1235
Name: count, dtype: int64

In [7]:
df = pd.concat([train,df_augmented],axis=0)
df['label'].value_counts()

label
0    15818
1    11235
Name: count, dtype: int64

In [8]:
test = pd.read_csv('test.csv')
df_test = test.copy()

# DataLoader

In [9]:
MAX_LEN = 512

class TextDataset(Dataset):
    def __init__(self, sentence, label, model_name, model_path, truncation_side):
        self.sentence = sentence
        self.label = label
        
        if model_name == "funnel-kor-base":
            self.tokenizer = FunnelTokenizerFast.from_pretrained(model_path)
        else:
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        if truncation_side == "left":
            self.tokenizer.truncation_side = 'left'

    def __len__(self): # 배치 생성을 위한 len 생성
        return len(self.sentence)

    def __getitem__(self, idx): # 생성된 배치를 호출하기 위한 getitem 생성
        sentence = self.sentence[idx]

        encoded = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,
            max_length = MAX_LEN,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
            return_token_type_ids=True,
            return_attention_mask=True,
        )
        
        input_ids = encoded["input_ids"][0] # 문장 -> 임베딩
        token_type_ids = encoded["token_type_ids"][0] # 문장이 2개일 때 구분
        attention_masks = encoded["attention_mask"][0] # 패딩된 토큰 무시
        
        if self.label is not None:
            label = self.label[idx]
            return [input_ids, token_type_ids, attention_masks], label
        else:
            return [input_ids, token_type_ids, attention_masks]

`input_ids         → Tensor [16, 512]`

`token_type_ids    → Tensor [16, 512]`

`attention_mask    → Tensor [16, 512]`  

`labels            → Tensor [16]`

# backbone model

In [10]:
latent_dim = model_dict['funnel-kor-base']['latent_dim']

class SwiGLU(nn.Module):
    def forward(self, x):
        x, gate = x.chunk(2, dim=-1)
        return F.silu(gate) * x

class TextModel(nn.Module):
    def __init__(self, model_name, model_path, latent_dim=latent_dim):
        super().__init__()
        if model_name == "funnel-kor-base":
            self.txt_model = FunnelModel.from_pretrained(model_path)
        else:
            self.txt_model = AutoModel.from_pretrained(model_path)
        # 분류기 정의
        self.classifier = nn.Sequential(
            SwiGLU(),
            nn.Linear(latent_dim, 1, bias=False)
        )

    def forward(self, x): # x는 튜플 형태의 입력 (input_ids, token_type_ids, attention_mask)
        input_ids = x[0]
        token_type_ids = x[1]
        attention_mask = x[2]

        # 입력 -> 벡터
        txt_side = self.txt_model( 
            input_ids=input_ids,
            token_type_ids=token_type_ids,
            attention_mask=attention_mask,
        )
        
        txt_feature = txt_side.last_hidden_state[:, 0, :] # [CLS] 토큰만 가져옴
        # -> 문장 분류 Task에서 [CLS] 토큰에 문장 전체 정보를 요약해서 담는 방식임.
        output = self.classifier(txt_feature)
        
        return output

`입력: x[0], x[1], x[2] -> logit`

# Main model

In [11]:
class TextClassifier(pl.LightningModule):
    def __init__(self, backbone, hparams):
        super().__init__() 
        self.backbone = backbone
        self.save_hyperparameters(hparams)

        if self.hparams.get('pos_weight') is not None:
            self.pos_weight = torch.tensor(self.hparams.pos_weight)

    def forward(self, x):
        predictions = self.backbone(x)
        return predictions

    def step(self, batch):
        # step 1
        x, y = batch
        # step 2
        y_hat = self.backbone(x)
        # step3
        if hasattr(self, "pos_weight"):
            loss_fn = nn.BCEWithLogitsLoss(pos_weight=self.pos_weight)
        else:
            loss_fn = nn.BCEWithLogitsLoss() 
        loss = loss_fn(y_hat.view(-1), y.float().view(-1))
        return loss, y, y_hat
    
    def training_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
    
        self.log("train_loss", loss, on_step=False, on_epoch=True, prog_bar=True)
        self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
    
        acc = (predictions.squeeze() == y).float().mean()
    
        # F1 score 계산
        f1 = f1_score(y.cpu(), predictions.cpu(), average='macro')
    
        self.log("val_loss", loss, on_epoch=True, prog_bar=True)
        self.log("val_acc", acc, on_epoch=True, prog_bar=True)
        self.log("val_f1", f1, on_epoch=True, prog_bar=True) 
        return {"val_loss": loss, "val_f1": f1}


    # predict_step과 달리 test set에서 성능 측정을 위한 함수
    def test_step(self, batch, batch_idx):
        loss, y, y_hat = self.step(batch)
        predictions = (y_hat > 0).float()
        acc = (predictions.squeeze() == y).float().mean()
    
        self.log("test_acc", acc)
    
    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        y_hat = self.backbone(batch)
        return y_hat
    
    # optimizer, scheduler가 없으면 학습이 안되기에 자동으로 찾아서 호출함.
    def configure_optimizers(self):
        opt_name = self.hparams.optimizer
        lr = self.hparams.learning_rate
    
        # 1. Optimizer 선택
        if opt_name == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=lr, momentum=0.9)
        elif opt_name == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        elif opt_name == "adamw":
            optimizer = torch.optim.AdamW(self.parameters(), lr=lr)
        else:
            raise ValueError(f"Unknown optimizer: {opt_name}")
    
        # 2. Scheduler 선택
        sched_name = self.hparams.scheduler
    
        if sched_name == "step":
            scheduler = torch.optim.lr_scheduler.StepLR(
                optimizer,
                step_size=2,
                gamma=0.9,
            )
    
        elif sched_name == "cosine":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=self.hparams.epochs,
                eta_min=1e-5,
            )
    
        elif sched_name == "onecyclelr":
            steps_per_epoch = getattr(self.hparams, "steps_per_epoch", 100)
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=lr,
                epochs=self.hparams.epochs,
                steps_per_epoch=steps_per_epoch,
                pct_start=0.1,
            )
        else:
            raise ValueError(f"Unknown scheduler: {sched_name}")
    
        return [optimizer], [scheduler]

# 학습

In [17]:
X = df["text"].values
y = df["label"].values
X_test = df_test['text'].values

optimizers = ["adamw"]
schedulers = ["cosine"]
truncation_side = 'left'
BATCH_SIZE = 16

CV = 3
skf = StratifiedKFold(n_splits=CV, shuffle=True, random_state=SEED)


result = []
for selected_model in model_dict.keys():
    model_path = model_dict[selected_model]["model_name"]
    latent_dim = model_dict[selected_model]["latent_dim"]

    for optimizer in optimizers:
        for scheduler in schedulers:
            val_acc_list = []
            preds_list = []

            for i, (train_index, val_index) in enumerate(skf.split(X, y)):
                X_train, X_val = X[train_index], X[val_index]
                y_train, y_val = y[train_index], y[val_index]

                train_ds = TextDataset(X_train, y_train, selected_model, model_path, truncation_side)
                val_ds = TextDataset(X_val, y_val, selected_model, model_path, truncation_side)
                test_ds = TextDataset(X_test, None, selected_model, model_path, truncation_side) # 정답 없음.

                train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
                val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE)
                test_loader = DataLoader(test_ds, batch_size=BATCH_SIZE)

                backbone = TextModel(model_name=selected_model, model_path=model_path, latent_dim=latent_dim)

                early_stop_cb = EarlyStopping(
                    monitor='val_f1',      # 모니터링할 지표
                    mode='max',            # 클수록 좋음 → max
                    patience=2,            # 개선 없으면 2번 기다리고 종료
                    verbose=True
                )

                steps_per_epoch = int(len(train_index) / BATCH_SIZE)
                hparams = {
                    "optimizer": optimizer,
                    "scheduler": scheduler, 
                    'steps_per_epoch': steps_per_epoch,
                    'output_dir': "output",  # 모델 체크포인트와 로그 파일을 저장할 디렉토리
                    'per_device_train_batch_size': BATCH_SIZE,  # 훈련 배치 크기
                    'per_device_eval_batch_size': BATCH_SIZE,  # 평가 배치 크기
                    'learning_rate': 0.00003,  # 학습률
                    'epochs' : 5,
                    'weight_decay': 0.01,  # 가중치 감쇠
                    'logging_dir': 'logs',  # 로그 파일을 저장할 디렉토리
                    'logging_steps': 100,  # 로그를 기록할 주기
                    'save_steps': 100,  # 모델을 저장할 주기
                    'evaluation_strategy': "steps",  # 평가 전략
                    'save_strategy': "steps",  # 저장 전략
                    'load_best_model_at_end': True,  # 최고의 모델을 끝에서 로드
                    'fp16': True,  # mixed precision (16-bit float)
                    'seed': SEED,  # 랜덤 시드 설정
                    'pos_weight': None,               
                }

                model = TextClassifier(backbone=backbone, hparams=hparams)

                checkpoint_cb = pl.callbacks.ModelCheckpoint(
                    dirpath="saved/",
                    filename=f"{selected_model}_{optimizer}_{scheduler}_fold{i}-{{epoch:02d}}-{{val_f1:.4f}}",
                    monitor="val_f1",
                    mode="max"
                )

                trainer = pl.Trainer(
                    max_epochs=5,
                    precision=16,
                    accelerator="auto",
                    devices=1,
                    callbacks=[checkpoint_cb, early_stop_cb]
                )

                trainer.fit(model, train_loader, val_loader)
                val_result = trainer.validate(model, dataloaders=val_loader)[0]

                y_preds = trainer.predict(model, dataloaders=test_loader)
                test_logits = torch.vstack(y_preds).cpu().numpy()

                val_acc_list.append(val_result["val_f1"])
                preds_list.append(test_logits)

                del model
                del trainer
                torch.cuda.empty_cache()
                gc.collect()
                
            # fold별 평균 저장
            result.append({
                'model': selected_model,
                'optimizer': optimizer,
                'scheduler': scheduler,
                'val_acc_mean': np.mean(val_acc_list),
                'logits': np.mean(preds_list, axis=0)
            })

In [14]:
result

[{'model': 'bert-kor-base',
  'optimizer': 'adamw',
  'scheduler': 'cosine',
  'val_acc_mean': 0.9845399657885233,
  'logits': array([[-10.55 ],
         [-10.86 ],
         [-10.62 ],
         ...,
         [ 11.266],
         [-10.9  ],
         [ 11.25 ]], dtype=float16)},
 {'model': 'kc-electra',
  'optimizer': 'adamw',
  'scheduler': 'cosine',
  'val_acc_mean': 0.9851706624031067,
  'logits': array([[-9.016],
         [-9.016],
         [-8.82 ],
         ...,
         [ 9.13 ],
         [-8.75 ],
         [ 8.67 ]], dtype=float16)},
 {'model': 'funnel-kor-base',
  'optimizer': 'adamw',
  'scheduler': 'cosine',
  'val_acc_mean': 0.9855538010597229,
  'logits': array([[-10.79 ],
         [-10.48 ],
         [-10.65 ],
         ...,
         [  9.555],
         [-10.67 ],
         [  9.12 ]], dtype=float16)}]

In [36]:
logits_list = [np.array(entry['logits']) for entry in result]
mean_logits = np.mean(logits_list, axis=0)
final_preds = (mean_logits > 0.2).astype(int).flatten()

pd.Series(final_preds).value_counts()

0    3514
1    3000
Name: count, dtype: int64

# Submission

In [33]:
# 병합 + 원래 순서로 정렬
sub = pd.read_csv('sample_submission.csv')
sub['label'] = final_preds

# 저장
sub.to_csv("submission44.csv", index=False)

---